<a href="https://colab.research.google.com/github/chihoang/Springboard_mec-mini-projects/blob/main/Mini_Project_SQL_with_Spark_Colab_good.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [ ]:
## install pyspark 
!pip install pyspark

# # Import SparkSession
# from pyspark.sql import SparkSession

# # Create a Spark Session
# spark = SparkSession.builder.master("local[*]").getOrCreate()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 59.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d445e882ce9ef278f5fee23667578fddb67e64929f5c20418b2a71461a99e89a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [ ]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
# Check Spark Session Information
spark

In [ ]:
sqlContext

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [ ]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [ ]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

,namespace
0,country_club
1,default


# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [ ]:
### 11/28/22: mount to the datasets in Google Drive ###
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'myfolder')
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


In [ ]:
### File location and type 
# file_location_bookings = "./Bookings.csv"
# file_location_facilities = "./Facilities.csv"
# file_location_members = "./Members.csv"

file_location_bookings = \
  "/content/gdrive/MyDrive/Colab Notebooks/mec-mini-projects-master/mec-5.6.6-sql-with-spark-mini-project/Bookings.csv"
file_location_facilities = \
  "/content/gdrive/MyDrive/Colab Notebooks/mec-mini-projects-master/mec-5.6.6-sql-with-spark-mini-project/Facilities.csv"
file_location_members = \
  "/content/gdrive/MyDrive/Colab Notebooks/mec-mini-projects-master/mec-5.6.6-sql-with-spark-mini-project/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [ ]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [ ]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [ ]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

,namespace,tableName,isTemporary
0,country_club,bookings,False
1,country_club,facilities,False
2,country_club,members,False


# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [ ]:
result = run_sql('''
select * 
from bookings limit 3
''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2


# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [ ]:
result = run_sql('''
select distinct name as paid_facilities
from facilities 
where membercost > 0 
                 ''')
result.toPandas()

,paid_facilities
0,Tennis Court 2
1,Squash Court
2,Tennis Court 1
3,Massage Room 1
4,Massage Room 2


####  Q2: How many facilities do not charge a fee to members?

In [ ]:
result = run_sql('''
select count(distinct name) as num_free_facilities
from facilities 
where membercost = 0 
                 ''')
result.toPandas()

,num_free_facilities
0,4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
result = run_sql('''
select facid, name, membercost, monthlymaintenance
from Facilities
where membercost < (0.2*monthlymaintenance) 
                ''')
result.toPandas()

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000
5,5,Massage Room 2,9.9,3000
6,6,Squash Court,3.5,80
7,7,Snooker Table,0.0,15
8,8,Pool Table,0.0,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [ ]:
result = run_sql('''
select * 
from facilities 
where facid in (1,5) 
                ''')
result.toPandas()

# where facid in ('1','5') 

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [ ]:
result = run_sql('''
select name, monthlymaintenance,
  case when monthlymaintenance <= 100 then 'cheap'
  when monthlymaintenance>100 then 'expensive' end as afford  
from facilities 
                 ''')
result.toPandas()

  # else 'expensive' end as affordability

,name,monthlymaintenance,afford
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [ ]:
result = run_sql('''
select firstname, surname
from bookings
  join members on bookings.memid = members.memid
where starttime = 
  (select max(starttime) from bookings) and 
      bookings.memid != '0'
                 ''')
result.toPandas()

,firstname,surname
0,Henry,Worthington-Smyth


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [ ]:
result = run_sql('''
select distinct facilities.name, 
  concat(members.firstname, ' ', members.surname) as member_name
from bookings
  join facilities on bookings.facid = facilities.facid
  join members on bookings.memid = members.memid
where facilities.name like '%%Tennis Court%%'
order by member_name 
                 ''')
result.toPandas()

# select distinct facilities.name,
#                 concat(members.firstname,' ',members.surname) as member_name     
# from bookings
#   join facilities on bookings.facid = facilities.facid
#   join members on bookings.memid = members.memid
# where facilities.name like '%%Tennis Court%%'
# order by member_name

,name,member_name
0,Tennis Court 2,Anne Baker
1,Tennis Court 1,Anne Baker
2,Tennis Court 2,Burton Tracy
3,Tennis Court 1,Burton Tracy
4,Tennis Court 1,Charles Owen
5,Tennis Court 2,Charles Owen
6,Tennis Court 2,Darren Smith
7,Tennis Court 2,David Farrell
8,Tennis Court 1,David Farrell
9,Tennis Court 2,David Jones


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [ ]:
result = run_sql('''
select name, concat(members.firstname,' ', members.surname) as member_name, 
  case when bookings.memid=0 then facilities.guestcost*bookings.slots
       else facilities.membercost*bookings.slots end as cost 
from bookings
  join facilities on bookings.facid = facilities.facid
  join members on bookings.memid = members.memid
-- where (facilities.membercost>30) or (facilities.guestcost>30) and 
--   (bookings.starttime>='2012-09-14T00:00:00' and bookings.starttime<='2012-09-14T24:00:00' )
where bookings.starttime like '%%2012-09-14%%' and
 ((facilities.guestcost*bookings.slots > 30 and bookings.memid=0) or 
  (facilities.membercost*bookings.slots >30 and bookings.memid!=0))
order by cost desc
                 ''')
result.toPandas()

# select name, concat(firstname,' ',surname) as member_name,
#   case when bookings.memid = 0 then guestcost * slots
#        else membercost * slots end as cost 
# from bookings
#   join facilities on Bookings.facid = facilities.facid
#   join members on bookings.memid = members.memid
# where starttime like '%%2012-09-14%%' and
#   ((guestcost * slots > 30 and bookings.memid = 0) or
#    (membercost * slots> 30 and bookings.memid != 0))
# order by cost desc

,name,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
result = run_sql('''
select name, concat(firstname,' ', surname) as member_name, 
  case when memid=0 then guestcost*slots
  else membercost*slots end as cost 
from (select bookings.memid as memid, bookings.starttime, bookings.slots,
             facilities.name, facilities.membercost, facilities.guestcost,
             members.surname, members.firstname
  from bookings
  join facilities on bookings.facid = facilities.facid
  join members on bookings.memid = members.memid ) as T
where starttime like '%%2012-09-14%%' and
 ((guestcost*slots > 30 and memid=0) or 
  (membercost*slots >30 and memid!=0))
order by cost desc
                 ''')
result.toPandas()

# select name,
#   concat(firstname,' ',surname) as member_name,
#   case when memid = 0 then guestcost * slots
#   else membercost * slots end as cost 
# from (select bookings.memid as memid,
#             bookings.facid,
#             firstname,
#             surname,
#             slots,
#             membercost,
#             guestcost,
#             starttime,
#             name
#       from bookings
#       join facilities on bookings.facid = facilities.facid
#       join members on bookings.memid = members.memid) as T
# where starttime like '%%2012-09-14%%' and
#   ((guestcost * slots > 30 and memid = 0) or 
#    (membercost * slots> 30 and memid != 0))
# order by cost desc

,name,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [ ]:
result = run_sql('''
select facilities.name, 
  sum(case when bookings.memid=0 then facilities.guestcost*bookings.slots
      else facilities.membercost*bookings.slots end) as revenue
from bookings
  join facilities on bookings.facid = facilities.facid
group by facilities.name
having revenue < 1000
order by revenue desc
                 ''')
result.toPandas()

# select name,
#     sum(fee * slots) as revenue
# from (select case when bookings.memid = 0 then guestcost
#              else membercost end as fee,
#              slots,
#              name
#       from bookings
#         join facilities on bookings.facid = facilities.facid) as booking_Fees
# group by name
# having revenue < 1000
# order by revenue

,name,revenue
0,Pool Table,270.0
1,Snooker Table,240.0
2,Table Tennis,180.0
